In [ ]:
import json
import pandas as pd
import os
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pytz
import ipywidgets as widgets
from ipywidgets import DatePicker, Button, HBox
from matplotlib.lines import Line2D
from pytz import timezone
from IPython.display import display, clear_output
from ipyfilechooser import FileChooser
import numpy as np

csv_file_path = 'sleep_data.csv'

sleep_data = pd.read_csv(csv_file_path)

# 'end_time'と'start_time'をdatetime型に変換
sleep_data['end_time'] = pd.to_datetime(sleep_data['end_time'])
sleep_data['start_time'] = pd.to_datetime(sleep_data['start_time'])
sleep_data['in_bed_time'] = pd.to_datetime(sleep_data['in_bed_time'])

# リスト作成
# 各セッションの最終 'end_time' を取得
last_end_time = sleep_data.groupby('session_id')['end_time'].max()
# 日本時間に変換して日付だけを抽出
session_dates = last_end_time.dt.tz_localize('UTC').dt.tz_convert('Asia/Tokyo')

# session_id と session_date の辞書を作成
session_date_dict = session_dates.to_dict()

# 日付選択ウィジェット
date_picker = DatePicker(description='Select Date', disabled=False)

# ボタンの設定
button_prev = Button(description="Previous Day")
button_next = Button(description="Next Day")

# ボタンクリックのイベントハンドラー
def on_prev_clicked(b):
    date_picker.value = date_picker.value - pd.Timedelta(days=1) if date_picker.value else None

def on_next_clicked(b):
    date_picker.value = date_picker.value + pd.Timedelta(days=1) if date_picker.value else None

button_prev.on_click(on_prev_clicked)
button_next.on_click(on_next_clicked)

# ウィジェットを表示
display(HBox([button_prev, button_next]))
display(date_picker)

def plot_sleep_data(date, data):
    date = pd.to_datetime(date).date()  # 日付オブジェクトに変換
    fig, ax = plt.subplots(figsize=(20, 7))
    stage_height = {3: 5, 1: 4, 4: 3, 6: 2, 5: 1}
    stage_colors = {1: '#e0ffff', 3: '#ff5252', 4: '#03a9f4', 5: '#303f9f', 6: '#ab47bc'}

    # 特定の日付に属するsession_idをフィルタリング
    relevant_sessions = data[(data['start_time'].dt.tz_localize(None).dt.date <= date) & 
                             (data['end_time'].dt.tz_localize(None).dt.date >= date)]
    
    if not relevant_sessions.empty:
        for session_id, session_data in relevant_sessions.groupby('session_id'):
            for index, row in session_data.iterrows():
                start_pos = mdates.date2num(row['start_time'].tz_localize(None))  # 時間帯の情報を削除
                duration = mdates.date2num(row['end_time'].tz_localize(None)) - start_pos
                ax.bar(x=start_pos, height=stage_height[row['sleep_state']], width=duration,
                       color=stage_colors.get(row['sleep_state'], '#FFFFFF'), edgecolor='black',
                       align='edge', alpha=0.8)

        # X軸の設定
        ax.xaxis_date(tz=timezone('Asia/Tokyo'))
        ax.xaxis.set_major_locator(mdates.HourLocator(interval=1))
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M', tz=timezone('Asia/Tokyo')))
        ax.set_ylim(0, 6)
        ax.set_yticks([1, 2, 3, 4, 5])
        ax.set_yticklabels(['Deep Sleep', 'REM', 'Light Sleep', 'Awake', 'Out-of-bed'])
        ax.set_xlabel('Time of Day')
        plt.tight_layout()
        plt.show()
    else:
        print("No sleep data available for this date.")

# 日付変更時のイベントハンドラ
def on_date_change(change):
    if change['new'] is not None:
        plot_sleep_data(change['new'], sleep_data)

date_picker.observe(on_date_change, names='value')

In [ ]:
# 日付選択ウィジェット #これに難あり

date_picker = widgets.DatePicker(
    description='Select Date',
    disabled=False
)

# 前日と翌日に移動するボタンの作成
button_prev = widgets.Button(description="Previous Day")
button_next = widgets.Button(description="Next Day")

# ボタンのクリックイベントハンドラ
def on_prev_clicked(b):
    new_date = date_picker.value - pd.Timedelta(days=1)
    date_picker.value = new_date

def on_next_clicked(b):
    new_date = date_picker.value + pd.Timedelta(days=1)
    date_picker.value = new_date

button_prev.on_click(on_prev_clicked)
button_next.on_click(on_next_clicked)

# ボタンを表示
display(widgets.HBox([button_prev, button_next]))
display(date_picker)

def plot_sleep_data(date, data):
    date = pd.to_datetime(date).date()
    fig, ax = plt.subplots(figsize=(10, 7))
    stage_height = {3: 5, 1: 4, 4: 3, 6: 2, 5: 1}
    stage_colors = {1: '#e0ffff', 3: '#ff5252', 4: '#03a9f4', 5: '#303f9f', 6: '#ab47bc'}

    # 特定の日付に属するsession_idをフィルタリング
    relevant_sessions = data[(data['start_time'].dt.date <= date) & (data['end_time'].dt.date >= date)]
    
    if not relevant_sessions.empty:
        for session_id, session_data in relevant_sessions.groupby('session_id'):
            for index, row in session_data.iterrows():
                start_pos = mdates.date2num(row['start_time'])
                duration = mdates.date2num(row['end_time']) - start_pos
                ax.bar(x=start_pos, height=stage_height[row['sleep_state']], width=duration,
                       color=stage_colors.get(row['sleep_state'], '#FFFFFF'), edgecolor='black',
                       align='edge', alpha=0.8)

        # X軸の設定
        ax.xaxis_date(tz=pytz.timezone('Asia/Tokyo'))
        ax.xaxis.set_major_locator(mdates.HourLocator(interval=1))
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M', tz=pytz.timezone('Asia/Tokyo')))
        ax.set_ylim(0, 6)
        ax.set_yticks([1, 2, 3, 4, 5])
        ax.set_yticklabels(['Deep Sleep', 'REM', 'Light Sleep', 'Awake', 'Out-of-bed'])
        ax.set_xlabel('Time of Day')
        plt.tight_layout()
        plt.show()
    else:
        print("No sleep data available for this date.")

# 日付変更時のイベントハンドラ
def on_date_change(change):
    if change['new'] is not None:
        plot_sleep_data(change['new'], sleep_data)

date_picker.observe(on_date_change, names='value')